In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pandas as pd

# setting user, api key and access token
plotly.tools.set_credentials_file(username='WendyLiang', api_key='0Mqn1LRKjujs81acpAGt')
mapbox_access_token = 'pk.eyJ1Ijoid2VuZHlsaWFuZyIsImEiOiJjandhZGVzd3cwOHU0NGFuMHBzdHJvNHdhIn0.15zho30YkMBcHvlAVx4J9w'

# load data
df = pd.read_csv('./data/UNESCO.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2722 entries, 0 to 2721
Data columns (total 15 columns):
ID                             2722 non-null int64
Name in English                2722 non-null object
Name in French                 2699 non-null object
Name in Spanish                2701 non-null object
Countries                      2721 non-null object
Country codes alpha 3          2721 non-null object
ISO639-3 codes                 2458 non-null object
Degree of endangerment         2722 non-null object
Alternate names                1583 non-null object
Name in the language           27 non-null object
Number of speakers             2539 non-null float64
Sources                        2079 non-null object
Latitude                       2719 non-null float64
Longitude                      2719 non-null float64
Description of the location    1870 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 319.1+ KB


In [2]:
# drop rows with no coordinate info
df = df[pd.notnull(df['Latitude'])]
df = df[pd.notnull(df['Countries'])]

# drop informations that we don't need
df = pd.DataFrame({'Name in English': df['Name in English'],
                   'Countries': df['Countries'],
                   'Country codes alpha 3': df['Country codes alpha 3'],
                   'Degree of endangerment': df['Degree of endangerment'],
                   'Number of speakers': df['Number of speakers'],
                   'Latitude': df['Latitude'],
                   'Longitude': df['Longitude']
})

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2718 entries, 0 to 2721
Data columns (total 7 columns):
Name in English           2718 non-null object
Countries                 2718 non-null object
Country codes alpha 3     2718 non-null object
Degree of endangerment    2718 non-null object
Number of speakers        2535 non-null float64
Latitude                  2718 non-null float64
Longitude                 2718 non-null float64
dtypes: float64(3), object(4)
memory usage: 169.9+ KB


,Name in English,Countries,Country codes alpha 3,Degree of endangerment,Number of speakers,Latitude,Longitude
0,South Italian,Italy,ITA,Vulnerable,7500000.0,40.9798,15.2490
1,Sicilian,Italy,ITA,Vulnerable,5000000.0,37.4399,14.5019
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...","DEU, DNK, NLD, POL, RUS",Vulnerable,4800000.0,53.4029,10.3601
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...","BRB, LVA, LTU, POL, RUS, UKR",Vulnerable,4000000.0,53.9560,27.5756
4,Lombard,"Italy, Switzerland","ITA, CHE",Definitely endangered,3500000.0,45.7215,9.3273


In [3]:
# select US data
usa = df.loc[df['Country codes alpha 3'] == 'USA']
usa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 156 to 2712
Data columns (total 7 columns):
Name in English           216 non-null object
Countries                 216 non-null object
Country codes alpha 3     216 non-null object
Degree of endangerment    216 non-null object
Number of speakers        207 non-null float64
Latitude                  216 non-null float64
Longitude                 216 non-null float64
dtypes: float64(3), object(4)
memory usage: 13.5+ KB


In [4]:
# degrees
degrees = ['Vulnerable', 'Definitely endangered', 'Severely endangered', 'Critically endangered', 'Extinct']

# colors
colors = ["rgb(0,255,255)", "rgb(0,250,154)", "rgb(255,215,0)", "rgb(255,140,0)", "rgb(255,69,0)"]

In [5]:
# get map data

data = []
for i in range(len(degrees)):
    degree = degrees[i]
    degree_data = go.Scattergeo(
        lat = usa.loc[usa['Degree of endangerment'] == degree, 'Latitude'],
        lon = usa.loc[usa['Degree of endangerment'] == degree, 'Longitude'],
        text = usa.loc[usa['Degree of endangerment'] == degree, 'Name in English'],
        mode = 'markers',
        marker = go.scattergeo.Marker(
            size = 8,
            opacity = 0.5,
            color = colors[i]
        ),
        name = degree,
        hoverinfo = 'text',
        type = 'scattergeo'
    )
    data.append(degree_data)

In [6]:
# layout of the map
layout_usa = dict(
        title = 'US Endangered Languages with Degree of Endangerment', 
        geo = dict(
            scope='usa',
            showland = True,
            showcoastlines = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
)

In [7]:
updatemenus=list([
    # drop-down: select different degrees to visualize
    dict(
         buttons=list([
            dict(label = 'All Degrees',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True]}]),
            dict(label = 'Vulnerable',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False]}]),
            dict(label = 'Definitely Endangered',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False]}]),
             dict(label = 'Severely Endangered',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False]}]),
             dict(label = 'Critically Endangered',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False]}]),
             dict(label = 'Extinct',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True]}])
             
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.01,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#FFFFFF',
        bordercolor = '#000000',
        font = dict(size=11)
    )
])

# assign the list of dictionaries to the layout dictionary
layout_usa['updatemenus'] = updatemenus

In [8]:
figure_usa = go.Figure(data=data, layout=layout_usa)
py.iplot(figure_usa, filename='US_EL_degree')

/usr/local/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [9]:
plotly.offline.plot(figure_usa, filename='US_EL_Degree.html', auto_open=False)

'US_EL_Degree.html'